# Refine the initialization procedure for XRB with the binary_c backbone

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
import matplotlib.gridspec as gridspec
import sys

import corner
import time
import pickle
import acor

import binary_c
import xrb
from xrb.binary import binary_evolve
from xrb.src import stats
from xrb.SF_history import sf_history

from xrb.src.core import *
set_data_path("../data")

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [10]:

def run_test_binary():
    m1 = 12.0
    m2 = 6.0
    eccentricity = 0.41
    metallicity = 0.02
    time = 33.895
    orbital_period = 4530.0

    sn_kick_magnitude_1 = 0.0
    sn_kick_theta_1 = 0.0
    sn_kick_phi_1 = 0.0
    sn_kick_magnitude_2 = 0.0
    sn_kick_theta_2 = 0.0
    sn_kick_phi_2 = 0.0

    output = binary_c.run_binary(m1, m2, orbital_period, eccentricity, metallicity, time,
                                 sn_kick_magnitude_1, sn_kick_theta_1, sn_kick_phi_1,
                                 sn_kick_magnitude_2, sn_kick_theta_2, sn_kick_phi_2, 0, 0)

    m1_out, m2_out, orbital_separation_out, eccentricity_out, system_velocity, L_x, \
            time_SN_1, time_SN_2, time_cur, ktype_1, ktype_2, comenv_count, evol_hist = output

    evol_steps = evol_hist.split("\n")
    # evol_steps = evol_steps["JEFF" in evol_steps]
    
#     print evol_steps
    
    for s in evol_steps:
        if s.find("JEFF") == 0:
            print s


    return output[:-1]

In [11]:
run_test_binary()

(1.3901519284928516,
 6.027029416008766,
 480.15170231162534,
 0.29861657126400837,
 190.2322874318976,
 0.0,
 20.065328181940924,
 0.0,
 33.895,
 13,
 1,
 1)

In [2]:
start = time.time()

c.sf_scheme = "SMC"
sampler = stats.run_emcee_population(nburn=10000, nsteps=10000, nwalkers=80, binary_scheme='binary_c')

end = time.time()

print "Elapsed time:", end-start, "seconds"

Elapsed time: 22937.7498949 seconds


/home/andrews/Research/XRB/xrb/SF_history/sf_history.py:214: RuntimeWarning: invalid value encountered in log10
  return sfh[index](np.log10(t_b*1.0e6))


In [ ]:
pickle.dump(sampler, open("../data/SMC_sampler.obj", "wb"))

In [2]:
sampler = pickle.load(open("../data/SMC_sampler.obj", "rb"))

In [4]:
print sampler.chain.shape


var_names = ["M1","M2","a","e","v_k","theta_k","phi_k","RA","Dec","time"]

for i in np.arange(10):
    print var_names[i], acor.acor(sampler.flatchain.T[i,360000:380000])[0], acor.acor(sampler.flatchain.T[i])[0]

(80, 20000, 10)
M1 379.850331993 4668.80463638
M2 677.161025742 3981.76058153
a 788.796620423 4152.31612535
e 192.850236877 3642.52596855
v_k 753.679194848 2891.49557333
theta_k 591.369945843 3481.50613504
phi_k 162.619850521 2430.62163611
RA 813.155813973 3965.25318047
Dec 360.801737748 2909.39653321
time 917.588508163 3329.89002425


In [ ]:
for i in range(sampler.dim):
    plt.figure()
    for chain in sampler.chain[...,i]:
        plt.plot(chain, alpha=0.25, color='k', drawstyle='steps')
        
plt.show()

In [ ]:
c.sf_scheme = "SMC"

# Corner plot
fontProperties = {'family':'serif', 'serif':['Times New Roman'], 'weight':'normal', 'size':12}
ticks_font = font_manager.FontProperties(family='Times New Roman', style='normal', \
                                         weight='normal', stretch='normal', size=12)
plt.rc('font', **fontProperties)

fig, ax = plt.subplots(10,10, figsize=(10,10))


labels = [r"$M_{\rm 1, i}\ (M_{\odot})$", r"$M_{\rm 2, i}\ (M_{\odot})$", r"$a_{\rm i}\ (R_{\odot})$", \
          r"$e_{\rm i}$", r"$v_{\rm k, i}\ ({\rm km}\ {\rm s}^{-1})$", r"$\theta_{\rm k}\ ({\rm rad.})$", \
          r"$\phi_{\rm k}\ ({\rm rad.})$", r"$\alpha_{\rm i}\ ({\rm deg.})$", \
          r"$\delta_{\rm i}\ ({\rm deg.}) $", r"$t_{\rm i}\ ({\rm Myr})$"]
plt_range = ([5,35], [5.0,22], [0,5900], [0,1], [0,1200], [0,np.pi], [0,np.pi], [6,21], [-75,-71], [0,120])

hist2d_kwargs = {"plot_datapoints" : False}
fig = corner.corner(sampler.flatchain, fig=fig, labels=labels, range=plt_range, max_n_ticks=4, **hist2d_kwargs)

ra_out = sampler.flatchain.T[7]
dec_out = sampler.flatchain.T[8]
gs = gridspec.GridSpec(2, 2,
                       width_ratios=[3,2],
                       height_ratios=[2,3]
                       )
smc_plot, ax1 = sf_history.get_SMC_plot_polar(20, fig_in=fig, gs=gs[1], ra_dist=ra_out, dec_dist=dec_out, \
                                              dist_bins=30, xwidth=2.0, ywidth=2.0, xgrid_density=6)


ax1.set_position([0.55, 0.55, 0.3, 0.3])


# Shift axis labels
for i in np.arange(10):
    ax[i,0].yaxis.set_label_coords(-0.5, 0.5)
    ax[9,i].xaxis.set_label_coords(0.5, -0.5)


plt.subplots_adjust(bottom=0.07, left=0.07, top=0.97)

plt.savefig('../figures/population_smc.pdf', rasterized=True)
# plt.show()

In [10]:
print len(sampler.flatchain)

1600000


In [10]:
def get_evol_set(sampler):

    N = len(sampler.flatchain)
    N = 20000

#     evol_set = np.array([], dtype='S')
    evol_set = np.zeros(N, dtype='S150')

    for i, sample_2 in zip(np.arange(N), sampler.flatchain):

        if i > N: break
        
        sample = sampler.flatchain[i]    

        m1, m2, a, ecc, v_k, theta_k, phi_k, alpha, delta, time = sample
        P_orb = binary_evolve.A_to_P(m1, m2, a)


        m1_out, m2_out, A_out, e_out, v_sys, L_x_out, tsn1, tsn2, k1_out, k2_out, comenv_count, evol_hist = \
                        binary_c.run_binary(m1, m2, P_orb, ecc, 0.008, time, v_k, theta_k, phi_k, v_k, theta_k, phi_k, 1)

        evol_steps = evol_hist.split("\n")

        evol_line = ""
        n_comenv = "0"

        for s in evol_steps:

            if s.find("JEFF") == 0:
                data = s.split(" ")
                k1_before, k2_before, k1_after, k2_after = data[2], data[3], data[15], data[16]

                if n_comenv != data[28]:
                    evol_line = evol_line + "(" + str(k1_before) + "-" + str(k2_before) + ":CE:" + str(k1_after) + "-" + str(k2_after) +  ")"
                    n_comenv = data[28]
                else:
                    evol_line = evol_line + "(" + str(k1_before) + "-" + str(k2_before) + ":" + str(k1_after) + "-" + str(k2_after) +  ")"

        evol_set[i] = evol_line
#         evol_set = np.append(evol_set, evol_line)


    return evol_set


In [ ]:
evol_set = get_evol_set(sampler)

In [9]:
# print evol_set
print len(evol_set)
print evol_set.nbytes

print evol_set

10000
1500000
['(1-1:2-1)(2-1:4-1)(4-1:7-1)(7-1:8-1)(8-1:13-1)'
 '(1-1:2-1)(2-1:4-1)(4-1:7-1)(7-1:8-1)(8-1:13-1)'
 '(1-1:2-1)(2-1:4-1)(4-1:7-1)(7-1:8-1)(8-1:13-1)' ...,
 '(1-1:2-1)(2-1:3-1)(3-1:CE:7-1)(7-1:8-1)(8-1:13-1)(13-1:13-2)(13-2:14-2)(14-2:14-3)(14-3:14-4)(14-4:14-7)'
 '(1-1:2-1)(2-1:3-1)(3-1:CE:7-1)(7-1:8-1)(8-1:13-1)(13-1:13-2)(13-2:14-2)(14-2:14-3)(14-3:14-4)(14-4:14-7)'
 '(1-1:2-1)(2-1:3-1)(3-1:CE:7-1)(7-1:8-1)(8-1:13-1)(13-1:13-2)(13-2:14-2)(14-2:14-3)(14-3:14-4)(14-4:14-7)']


In [ ]:
pickle.dump(evol_set, open("../data/SMC_evol_set.obj", "wb"))

In [25]:
evol_set = pickle.load(open("../data/SMC_evol_set.obj", "rb"))

# hists, n_hists = np.unique(evol_set, return_counts=True)

# idx = np.argsort(n_hists)[::-1]
# for i in np.arange(len(hists)):
#     print n_hists[idx[i]], hists[idx[i]]


In [ ]:
def generate_corner_plot(posteriors, filename=False):

    # Corner plot
    fontProperties = {'family':'serif', 'serif':['Times New Roman'], 'weight':'normal', 'size':12}
    ticks_font = font_manager.FontProperties(family='Times New Roman', style='normal', \
                                             weight='normal', stretch='normal', size=12)
    plt.rc('font', **fontProperties)

    fig, ax = plt.subplots(10,10, figsize=(10,10))


    labels = [r"$M_{\rm 1, i}\ (M_{\odot})$", r"$M_{\rm 2, i}\ (M_{\odot})$", r"$a_{\rm i}\ (R_{\odot})$", \
              r"$e_{\rm i}$", r"$v_{\rm k, i}\ ({\rm km}\ {\rm s}^{-1})$", r"$\theta_{\rm k}\ ({\rm rad.})$", \
              r"$\phi_{\rm k}\ ({\rm rad.})$", r"$\alpha_{\rm i}\ ({\rm deg.})$", \
              r"$\delta_{\rm i}\ ({\rm deg.}) $", r"$t_{\rm i}\ ({\rm Myr})$"]
    plt_range = ([5,35], [2.5,22], [0,5900], [0,1], [0,1200], [0,np.pi], [0,np.pi], [6,21], [-75,-71], [0,120])

    hist2d_kwargs = {"plot_datapoints" : False}
    fig = corner.corner(posteriors, fig=fig, labels=labels, range=plt_range, max_n_ticks=4, **hist2d_kwargs)

    ra_out = posteriors.T[7]
    dec_out = posteriors.T[8]
    gs = gridspec.GridSpec(2, 2,
                           width_ratios=[3,2],
                           height_ratios=[2,3]
                           )
    smc_plot, ax1 = sf_history.get_SMC_plot_polar(40, fig_in=fig, gs=gs[1], ra_dist=ra_out, dec_dist=dec_out, \
                                                  dist_bins=30, xwidth=2.0, ywidth=2.0, xgrid_density=6)


    ax1.set_position([0.55, 0.55, 0.3, 0.3])


    # Shift axis labels
    for i in np.arange(10):
        ax[i,0].yaxis.set_label_coords(-0.5, 0.5)
        ax[9,i].xaxis.set_label_coords(0.5, -0.5)


    plt.subplots_adjust(bottom=0.07, left=0.07, top=0.97)

    if filename is True:
        plt.savefig(filename, rasterized=True)
    else:
        plt.show()
    

In [ ]:
# print evol_set[:].endswith("13-1)")

# Separate in CE vs. non CE systems
idx_non_CE = np.char.find(evol_set, "CE") == -1
idx_CE = np.char.find(evol_set, "CE") != -1
idx_CE_one = np.char.count(evol_set, "CE") == 1
idx_CE_two = np.char.count(evol_set, "CE") == 2
idx_CE_AGB = np.char.find(evol_set, "5-1:CE") != -1
idx_CE_GB = np.char.find(evol_set, "3-1:CE") != -1
idx_CE_postGB = np.logical_or(np.char.find(evol_set, "4-1:CE") != -1, np.char.find(evol_set, "5-1:CE") != -1)

evol_non_CE = sampler.flatchain[idx_non_CE]
evol_CE = sampler.flatchain[idx_CE]
evol_CE_one = sampler.flatchain[idx_CE_one]
evol_CE_two = sampler.flatchain[idx_CE_two]
evol_CE_AGB = sampler.flatchain[idx_CE_AGB]
evol_CE_GB = sampler.flatchain[idx_CE_GB]
evol_CE_postGB = sampler.flatchain[idx_CE_postGB]


# Donor types
idx_MS_donor = np.char.endswith(evol_set, "-1)")
idx_HG_donor = np.char.endswith(evol_set, "-2)")
idx_RG_donor = np.char.endswith(evol_set, "-3)")
idx_HB_donor = np.char.endswith(evol_set, "-4)")
idx_FAGB_donor = np.char.endswith(evol_set, "-5)")
idx_SAGB_donor = np.char.endswith(evol_set, "-6)")
idx_NMS_donor = np.char.endswith(evol_set, "-7)")
idx_NHG_donor = np.char.endswith(evol_set, "-8)")
idx_NGB_donor = np.char.endswith(evol_set, "-9)")
idx_HeWD_donor = np.char.endswith(evol_set, "-10)")
idx_COWD_donor = np.char.endswith(evol_set, "-11)")
idx_ONeWD_donor = np.char.endswith(evol_set, "-12)")

evol_MS_donor = sampler.flatchain[idx_MS_donor]
evol_HG_donor = sampler.flatchain[idx_HG_donor]
evol_RG_donor = sampler.flatchain[idx_RG_donor]
evol_HB_donor = sampler.flatchain[idx_HB_donor]
evol_FAGB_donor = sampler.flatchain[idx_FAGB_donor]
evol_SAGB_donor = sampler.flatchain[idx_SAGB_donor]
evol_NMS_donor = sampler.flatchain[idx_NMS_donor]
evol_NHG_donor = sampler.flatchain[idx_NHG_donor]
evol_NGB_donor = sampler.flatchain[idx_NGB_donor]
evol_HeWD_donor = sampler.flatchain[idx_HeWD_donor]
evol_COWD_donor = sampler.flatchain[idx_COWD_donor]
evol_ONeWD_donor = sampler.flatchain[idx_ONeWD_donor]


print "MS Donors:", len(evol_MS_donor)
print "HG Donors:", len(evol_HG_donor)
print "RG Donors:", len(evol_RG_donor)
print "HB Donors:", len(evol_HB_donor)
print "FAGB Donors:", len(evol_FAGB_donor)
print "SAGB Donors:", len(evol_SAGB_donor)
print "NMS Donors:", len(evol_NMS_donor)
print "NHG Donors:", len(evol_NHG_donor)
print "NGB Donors:", len(evol_NGB_donor)
print "HeWD Donors:", len(evol_HeWD_donor)
print "COWD Donors:", len(evol_COWD_donor)
print "ONeWD Donors:", len(evol_ONeWD_donor)
print ""


idx_MS_CE_donor = np.logical_and(np.char.endswith(evol_set, "-1)"), np.char.find(evol_set, "CE") != -1)
idx_MS_non_CE_donor = np.logical_and(np.char.endswith(evol_set, "-1)"), np.char.find(evol_set, "CE") == -1)
idx_HB_CE_donor = np.logical_and(np.char.endswith(evol_set, "-4)"), np.char.find(evol_set, "CE") != -1)
idx_HB_non_CE_donor = np.logical_and(np.char.endswith(evol_set, "-4)"), np.char.find(evol_set, "CE") == -1)
idx_NMS_CE_donor = np.logical_and(np.char.endswith(evol_set, "-7)"), np.char.find(evol_set, "CE") != -1)
idx_NMS_non_CE_donor = np.logical_and(np.char.endswith(evol_set, "-7)"), np.char.find(evol_set, "CE") == -1)

evol_MS_CE_donor = sampler.flatchain[idx_MS_CE_donor]
evol_MS_non_CE_donor = sampler.flatchain[idx_MS_non_CE_donor]
evol_HB_CE_donor = sampler.flatchain[idx_HB_CE_donor]
evol_HB_non_CE_donor = sampler.flatchain[idx_HB_non_CE_donor]
evol_NMS_CE_donor = sampler.flatchain[idx_NMS_CE_donor]
evol_NMS_non_CE_donor = sampler.flatchain[idx_NMS_non_CE_donor]



print "MS CE Donors:", len(evol_MS_CE_donor)
print "MS nonCE Donors:", len(evol_MS_non_CE_donor)
print "HB CE Donors:", len(evol_HB_CE_donor)
print "HB nonCE Donors:", len(evol_HB_non_CE_donor)
print "NMS CE Donors:", len(evol_NMS_CE_donor)
print "NMS nonCE Donors:", len(evol_NMS_non_CE_donor)


In [123]:
print len(evol_CE)
print len(evol_CE_one)
print len(evol_CE_two)

print len(evol_CE_AGB)
print len(evol_CE_GB)

194349
138295
56054
121826
154692


In [ ]:
generate_corner_plot(evol_CE_AGB, filename="../data/SMC_corner_CE_AGB.pdf")

In [ ]:
generate_corner_plot(evol_CE_GB, filename="../data/SMC_corner_CE_GB.pdf")

In [ ]:
generate_corner_plot(evol_CE_postGB, filename="../data/SMC_corner_CE_postGB.pdf")

In [ ]:
generate_corner_plot(evol_CE_two, filename="../data/SMC_corner_CE_two.pdf")

In [ ]:
generate_corner_plot(evol_non_CE, filename="../data/SMC_corner_non_CE.pdf")

In [108]:

hists, n_hists = np.unique(evol_set, return_counts=True)

idx = np.argsort(n_hists)[::-1]
for i in np.arange(len(hists)):
    print n_hists[idx[i]], hists[idx[i]]

82114 (1-1:2-1)(2-1:4-1)(4-1:7-1)(7-1:8-1)(8-1:13-1)
38922 (1-1:2-1)(2-1:4-1)(4-1:5-1)(5-1:CE:8-1)(8-1:13-1)
26420 (1-1:2-1)(2-1:3-1)(3-1:CE:7-1)(7-1:8-1)(8-1:13-1)
24184 (1-1:2-1)(2-1:4-1)(4-1:CE:7-1)(7-1:8-1)(8-1:13-1)
20898 (1-1:2-1)(2-1:3-1)(3-1:4-1)(4-1:5-1)(5-1:CE:8-1)(8-1:13-1)
16109 (1-1:2-1)(2-1:3-1)(3-1:4-1)(4-1:5-1)(5-1:CE:8-1)(8-1:13-1)(13-1:13-2)(13-2:13-3)(13-3:CE:13-7)
13461 (1-1:2-1)(2-1:3-1)(3-1:4-1)(4-1:7-1)(7-1:8-1)(8-1:13-1)
11276 (1-1:2-1)(2-1:4-1)(4-1:5-1)(5-1:CE:8-1)(8-1:13-1)(13-1:13-2)(13-2:13-3)(13-3:CE:13-7)
7091 (1-1:2-1)(2-1:3-1)(3-1:4-1)(4-1:5-1)(5-1:CE:8-1)(8-1:13-1)(13-1:13-2)(13-2:CE:13-7)
6452 (1-1:2-1)(2-1:3-1)(3-1:4-1)(4-1:5-1)(5-1:13-1)(13-1:13-2)(13-2:13-3)(13-3:13-4)
5463 (1-1:2-1)(2-1:4-1)(4-1:5-1)(5-1:CE:8-1)(8-1:13-1)(13-1:13-2)(13-2:CE:13-7)
4017 (1-1:2-1)(2-1:3-1)(3-1:4-1)(4-1:5-1)(5-1:13-1)
3693 (1-1:2-1)(2-1:4-1)(4-1:7-1)(7-1:8-1)(8-1:13-1)(13-1:13-2)(13-2:13-4)
3441 (1-1:2-1)(2-1:CE:7-1)(7-1:8-1)(8-1:13-1)
3310 (1-1:2-1)(2-1:4-1)(4-1:CE:7-